In [3]:
!nvidia-smi

Wed Jan 19 16:56:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   29C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.10.0 True


### i. Install mmcv-full

In [5]:
pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.10.0/index.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.10.0/index.html
     |████████████████████████████████| 69.6 MB 7.9 MB/s             
     |████████████████████████████████| 190 kB 28.3 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


### ii. register Segmind Hook manually

In [5]:
from mmcv.runner import HOOKS, LoggerHook, master_only

from segmind.tracking.fluent import log_metrics
from segmind.utils.logging_utils import try_mlflow_log


@HOOKS.register_module()
class SegmindLoggerHook(LoggerHook):
    """Class to log metrics to Segmind.
    It requires `Segmind`_ to be installed.
    .. _Segmind:
        https://docs.segmind.com/python-library
    """

    def __init__(self):
        super(SegmindLoggerHook, self).__init__()
        self.import_segmind()

    def import_segmind(self):
        try:
            import segmind
        except ImportError:
            raise ImportError(
                "Please run 'pip install segmind' to install segmind")
        self.segmind_log_metrics = segmind.tracking.fluent.log_metrics
        self.segmind_mlflow_log = segmind.utils.logging_utils.try_mlflow_log

    @master_only
    def log(self, runner):
        tags = self.get_loggable_tags(runner)
        if tags:
            # logging metrics to segmind
            self.segmind_mlflow_log(
                self.segmind_log_metrics,
                tags,
                step=runner.epoch,
                epoch=runner.epoch)

### ii. Run `mmcv/examples/train.py` (with Segmind Logger)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

from mmcv.parallel import MMDataParallel
from mmcv.runner import EpochBasedRunner
from mmcv.utils import get_logger


class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def train_step(self, data, optimizer):
        images, labels = data
        predicts = self(images)  # -> self.__call__() -> self.forward()
        loss = self.loss_fn(predicts, labels)
        return {'loss': loss}


if __name__ == '__main__':
    model = Model()
    if torch.cuda.is_available():
        # only use gpu:0 to train
        # Solved issue https://github.com/open-mmlab/mmcv/issues/1470
        model = MMDataParallel(model.cuda(), device_ids=[0])

    # dataset and dataloader
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    trainset = CIFAR10(
        root='data', train=True, download=True, transform=transform)
    trainloader = DataLoader(
        trainset, batch_size=128, shuffle=True, num_workers=2)

    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    logger = get_logger('mmcv')
    # runner is a scheduler to manage the training
    runner = EpochBasedRunner(
        model,
        optimizer=optimizer,
        work_dir='./work_dir',
        logger=logger,
        max_epochs=4)

    # learning rate scheduler config
    lr_config = dict(policy='step', step=[2, 3])
    # configuration of optimizer
    optimizer_config = dict(grad_clip=None)
    # configuration of saving checkpoints periodically
    checkpoint_config = dict(interval=1)
    # save log periodically and multiple hooks can be used simultaneously
    log_config = dict(interval=100, hooks=[dict(type='TextLoggerHook')])
    
    # register hooks to runner and those hooks will be invoked automatically
    custom_hooks = [dict(type="SegmindLoggerHook")]
    runner.register_training_hooks(
        lr_config=lr_config,
        optimizer_config=optimizer_config,
        checkpoint_config=checkpoint_config,
        log_config=log_config,
        custom_hooks_config=custom_hooks)

    runner.run([trainloader], [('train', 1)])


Files already downloaded and verified


2022-01-19 16:56:33,270 - mmcv - INFO - Start running, host: juno@nb-staging-387-65966bfd44-g7v5h, work_dir: /home/juno/work_dir
2022-01-19 16:56:33,271 - mmcv - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) SegmindLoggerHook                  
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) SegmindLoggerHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL      ) CheckpointHook                     
(NORMAL      ) SegmindLoggerHook  

endpoint:  /api/2.0/mlflow/runs/create {'method': 'POST', 'json': {'experiment_id': '511effe6-9d42-4856-8e67-c98ffbe94d91', 'tags': [{'key': 'cral.tracking.user', 'value': 'juno'}, {'key': 'cral.tracking.source.name', 'value': '/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py'}, {'key': 'cral.tracking.source.type', 'value': 'LOCAL'}]}}
endpoint:  /api/2.0/mlflow/runs/log-batch {'method': 'POST', 'json': {'run_id': '54d993fa893a4890a7eb84af33803069', 'metrics': [{'key': 'learning_rate', 'value': 0.001, 'timestamp': '1642611397617', 'step': '0', 'epoch': '0', 'tags': [{'key': 'sys_metric', 'value': 'no'}]}, {'key': 'momentum', 'value': 0.9, 'timestamp': '1642611397617', 'step': '0', 'epoch': '0', 'tags': [{'key': 'sys_metric', 'value': 'no'}]}], 'tags': [{'key': 'sys_metric', 'value': 'no'}]}}


2022-01-19 16:56:39,187 - mmcv - INFO - Epoch [1][10/391]	lr: 1.000e-03, eta: 1:18:51, time: 0.305, data_time: 0.217, memory: 14, 


endpoint:  /api/2.0/mlflow/runs/log-batch {'method': 'POST', 'json': {'run_id': '54d993fa893a4890a7eb84af33803069', 'metrics': [{'key': 'learning_rate', 'value': 0.001, 'timestamp': '1642611399302', 'step': '0', 'epoch': '0', 'tags': [{'key': 'sys_metric', 'value': 'no'}]}, {'key': 'momentum', 'value': 0.9, 'timestamp': '1642611399302', 'step': '0', 'epoch': '0', 'tags': [{'key': 'sys_metric', 'value': 'no'}]}], 'tags': [{'key': 'sys_metric', 'value': 'no'}]}}


...

2022-01-19 17:00:55,365 - mmcv - INFO - Epoch [4][370/391]	lr: 1.000e-05, eta: 0:00:35, time: 0.164, data_time: 0.007, memory: 14, 


endpoint:  /api/2.0/mlflow/runs/log-batch {'method': 'POST', 'json': {'run_id': '54d993fa893a4890a7eb84af33803069', 'metrics': [{'key': 'learning_rate', 'value': 1.0000000000000003e-05, 'timestamp': '1642611655480', 'step': '3', 'epoch': '3', 'tags': [{'key': 'sys_metric', 'value': 'no'}]}, {'key': 'momentum', 'value': 0.9, 'timestamp': '1642611655480', 'step': '3', 'epoch': '3', 'tags': [{'key': 'sys_metric', 'value': 'no'}]}], 'tags': [{'key': 'sys_metric', 'value': 'no'}]}}


2022-01-19 17:00:56,948 - mmcv - INFO - Epoch [4][380/391]	lr: 1.000e-05, eta: 0:00:18, time: 0.161, data_time: 0.007, memory: 14, 


endpoint:  /api/2.0/mlflow/runs/log-batch {'method': 'POST', 'json': {'run_id': '54d993fa893a4890a7eb84af33803069', 'metrics': [{'key': 'learning_rate', 'value': 1.0000000000000003e-05, 'timestamp': '1642611657070', 'step': '3', 'epoch': '3', 'tags': [{'key': 'sys_metric', 'value': 'no'}]}, {'key': 'momentum', 'value': 0.9, 'timestamp': '1642611657070', 'step': '3', 'epoch': '3', 'tags': [{'key': 'sys_metric', 'value': 'no'}]}], 'tags': [{'key': 'sys_metric', 'value': 'no'}]}}


2022-01-19 17:00:58,703 - mmcv - INFO - Epoch [4][390/391]	lr: 1.000e-05, eta: 0:00:01, time: 0.159, data_time: 0.008, memory: 14, 
2022-01-19 17:00:58,757 - mmcv - INFO - Saving checkpoint at 4 epochs
